In [12]:
from snowflake.snowpark import Session
from snowflake.core import Root
# from snowflake.core.database import Database
# from snowflake.core.schema import Schema 

In [13]:
session = Session.builder.config("connection_name","myconnection").create()
root = Root(session)

In [14]:
## Load the info table FROM the database into a Snowflake DataFrame
sp500_info_df = session.table("investment_db.investment_schema.sp500_info")
sp500_info_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## SELECTING DATA

In [16]:
## How to filter the data (i.e. WHERE clause)
from snowflake.snowpark.functions import col
sp500_info_df_AMZN = sp500_info_df.filter(col('SYMBOL') == "AMZN")
sp500_info_df_AMZN.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
## How to select specific columns (i.e. SELECT clause) - FOUR METHODS
sp500_info_df_SELECT = sp500_info_df.select(col('SYMBOL'), col('LONGNAME'), col('SECTOR'), col('INDUSTRY'), col('COL_52WEEKCHANGE'), col('SANDP52WEEKCHANGE'))

# sp500_info_df_SELECT = sp500_info_df.select('SYMBOL', 'LONGNAME', 'SECTOR', 'INDUSTRY', 'COL_52WEEKCHANGE', 'SANDP52WEEKCHANGE')

# sp500_info_df_SELECT = sp500_info_df.select(sp500_info_df['SYMBOL'], sp500_info_df['LONGNAME'], sp500_info_df['SECTOR'], sp500_info_df['INDUSTRY'], sp500_info_df['COL_52WEEKCHANGE'], sp500_info_df['SANDP52WEEKCHANGE'])

# sp500_info_df_SELECT = sp500_info_df.select(sp500_info_df.SYMBOL, sp500_info_df.LONGNAME, sp500_info_df.SECTOR, sp500_info_df.INDUSTRY, sp500_info_df.COL_52WEEKCHANGE, sp500_info_df.SANDP52WEEKCHANGE)

sp500_info_df_SELECT.show()

---------------------------------------------------------------------------------------------------------------------------------------------
|"SYMBOL"  |"LONGNAME"                    |"SECTOR"            |"INDUSTRY"                       |"COL_52WEEKCHANGE"  |"SANDP52WEEKCHANGE"  |
---------------------------------------------------------------------------------------------------------------------------------------------
|MMM       |3M Company                    |Industrials         |Conglomerates                    |-0.12404817         |0.23614728           |
|AOS       |A. O. Smith Corporation       |Industrials         |Specialty Industrial Machinery   |0.41277206          |0.23614728           |
|ABT       |Abbott Laboratories           |Healthcare          |Medical Devices                  |0.0065672398        |0.23614728           |
|ABBV      |AbbVie Inc.                   |Healthcare          |Drug Manufacturers - General     |-0.046132863        |0.23614728           |
|ACN  

## JOINING DATA

In [ ]:
from snowflake.snowpark.functions import col

In [22]:
## Load the returns table FROM the database into a Snowflake DataFrame
sp500_returns_df = session.table("investment_db.investment_schema.sp500_returns")
sp500_returns_df.show()

-------------------------------------------------------------------------------------------
|"DATE"      |"TICKER"  |"RETURN"             |"RETURN_SP500"       |"EXCESS_RETURN"      |
-------------------------------------------------------------------------------------------
|2020-01-02  |A         |NULL                 |NULL                 |NULL                 |
|2020-01-03  |A         |-0.0160558625376078  |-0.0070598705620453  |-0.0089959919755624  |
|2020-01-06  |A         |0.0029560971447193   |0.0035333728907271   |-0.0005772757460078  |
|2020-01-07  |A         |0.0030655935136134   |-0.0028032386528962  |0.0058688321665096   |
|2020-01-08  |A         |0.0098730003124209   |0.0049024513689781   |0.0049705489434428   |
|2020-01-09  |A         |0.0157120164791473   |0.0066552626054889   |0.0090567538736583   |
|2020-01-10  |A         |0.0036669906801432   |-0.0028551786896625  |0.0065221693698057   |
|2020-01-13  |A         |-0.0014838309342424  |0.0069762152525698   |-0.00846004

In [23]:
## View the info table we loaded earlier
sp500_info_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [24]:
## Join the info and returns tables
sp500_joined_dfs = sp500_returns_df.join(sp500_info_df, col('TICKER') == col('SYMBOL'))
sp500_joined_dfs.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [44]:
## Join with filter -- shows the return for 2023-12-14 for each stock
sp500_joined_dfs = sp500_returns_df.join(
    sp500_info_df, 
    (sp500_returns_df.col("TICKER") == sp500_info_df.col("SYMBOL")) & (sp500_returns_df.col("DATE") == '2023-12-14')).select(
        "DATE", sp500_returns_df["TICKER"].as_("TICKER"), "RETURN", "RETURN_SP500", "LONGNAME").show()

---------------------------------------------------------------------------------------------------
|"DATE"      |"TICKER"  |"RETURN"             |"RETURN_SP500"      |"LONGNAME"                    |
---------------------------------------------------------------------------------------------------
|2023-12-14  |MMM       |0.0273538574463885   |0.0026470624846992  |3M Company                    |
|2023-12-14  |AOS       |0.0133282732913819   |0.0026470624846992  |A. O. Smith Corporation       |
|2023-12-14  |ABT       |0.0125873983601034   |0.0026470624846992  |Abbott Laboratories           |
|2023-12-14  |ABBV      |0.0037589230044288   |0.0026470624846992  |AbbVie Inc.                   |
|2023-12-14  |ACN       |-0.0026481306696496  |0.0026470624846992  |Accenture plc                 |
|2023-12-14  |ADBE      |-0.0634671362851876  |0.0026470624846992  |Adobe Inc.                    |
|2023-12-14  |AMD       |-0.001374936233081   |0.0026470624846992  |Advanced Micro Devices, Inc.  |


In [45]:
## Create a new column based on a calculation
sp500_calced = sp500_returns_df.select(
    col('DATE'),
    col('TICKER'), 
    col('RETURN'), 
    col('RETURN_SP500'), 
    (col('RETURN') - col('RETURN_SP500')).as_('EXCESS_RETURN')
    )
sp500_calced.show()

----------------------------------------------------------------------------------------------
|"DATE"      |"TICKER"  |"RETURN"             |"RETURN_SP500"       |"EXCESS_RETURN"         |
----------------------------------------------------------------------------------------------
|2020-01-02  |A         |NULL                 |NULL                 |NULL                    |
|2020-01-03  |A         |-0.0160558625376078  |-0.0070598705620453  |-0.008995991975562499   |
|2020-01-06  |A         |0.0029560971447193   |0.0035333728907271   |-0.0005772757460078002  |
|2020-01-07  |A         |0.0030655935136134   |-0.0028032386528962  |0.0058688321665096      |
|2020-01-08  |A         |0.0098730003124209   |0.0049024513689781   |0.0049705489434427996   |
|2020-01-09  |A         |0.0157120164791473   |0.0066552626054889   |0.0090567538736584      |
|2020-01-10  |A         |0.0036669906801432   |-0.0028551786896625  |0.0065221693698057      |
|2020-01-13  |A         |-0.0014838309342424  |0.0

In [46]:
## Chaining Method calls
sp500_joined_calced_filtered_selected = sp500_returns_df.select(
    col('DATE'),
    col('TICKER'), 
    col('RETURN'), 
    col('RETURN_SP500'), 
    (col('RETURN') - col('RETURN_SP500')).as_('EXCESS_RETURN')
    ).filter(col('TICKER') == 'AMZN').select('DATE', col('TICKER'), col('EXCESS_RETURN')).show()

--------------------------------------------------
|"DATE"      |"TICKER"  |"EXCESS_RETURN"         |
--------------------------------------------------
|2020-01-02  |AMZN      |NULL                    |
|2020-01-03  |AMZN      |-0.005079164093875601   |
|2020-01-06  |AMZN      |0.0113522010397793      |
|2020-01-07  |AMZN      |0.0048948590685355      |
|2020-01-08  |AMZN      |-0.0127110916302674     |
|2020-01-09  |AMZN      |-0.0018560878190205     |
|2020-01-10  |AMZN      |-0.0065554181123687     |
|2020-01-13  |AMZN      |-0.0026536370068704004  |
|2020-01-14  |AMZN      |-0.0100436789452504     |
|2020-01-15  |AMZN      |-0.005839323433436999   |
--------------------------------------------------



In [49]:
## Retrieve the column definitions for a table by using the schema property of the table object.
from snowflake.snowpark.types import *
# Get the StructType object that describes the columns in the underlying rowset.
table_schema = session.table("investment_db.investment_schema.sp500_info").schema
table_schema

StructType([StructField('SYMBOL', StringType(16777216), nullable=True), StructField('ADDRESS1', StringType(16777216), nullable=True), StructField('CITY', StringType(16777216), nullable=True), StructField('STATE', StringType(16777216), nullable=True), StructField('ZIP', StringType(16777216), nullable=True), StructField('COUNTRY', StringType(16777216), nullable=True), StructField('PHONE', StringType(16777216), nullable=True), StructField('WEBSITE', StringType(16777216), nullable=True), StructField('INDUSTRY', StringType(16777216), nullable=True), StructField('INDUSTRYKEY', StringType(16777216), nullable=True), StructField('INDUSTRYDISP', StringType(16777216), nullable=True), StructField('SECTOR', StringType(16777216), nullable=True), StructField('SECTORKEY', StringType(16777216), nullable=True), StructField('SECTORDISP', StringType(16777216), nullable=True), StructField('LONGBUSINESSSUMMARY', StringType(16777216), nullable=True), StructField('FULLTIMEEMPLOYEES', DoubleType(), nullable=Tr

## Performing an Action to Evaluate a DataFrame

In [52]:
## collect() method returns an array of Row objects
sp500_info_df.collect()

[Row(SYMBOL='MMM', ADDRESS1='3M Center', CITY='Saint Paul', STATE='MN', ZIP='55144-1000', COUNTRY='United States', PHONE='651 733 1110', WEBSITE='https://www.3m.com', INDUSTRY='Conglomerates', INDUSTRYKEY='conglomerates', INDUSTRYDISP='Conglomerates', SECTOR='Industrials', SECTORKEY='industrials', SECTORDISP='Industrials', LONGBUSINESSSUMMARY='3M Company provides diversified technology services in the United States and internationally. The company operates through four segments: Safety and Industrial; Transportation and Electronics; Health Care; and Consumer. The Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; closure systems for personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection sol

In [53]:
## count() method returns the number of rows in the DataFrame
sp500_info_df.count()

503

In [60]:
## first() method returns the first row in the DataFrame
sp500_info_df.first()

Row(SYMBOL='MMM', ADDRESS1='3M Center', CITY='Saint Paul', STATE='MN', ZIP='55144-1000', COUNTRY='United States', PHONE='651 733 1110', WEBSITE='https://www.3m.com', INDUSTRY='Conglomerates', INDUSTRYKEY='conglomerates', INDUSTRYDISP='Conglomerates', SECTOR='Industrials', SECTORKEY='industrials', SECTORDISP='Industrials', LONGBUSINESSSUMMARY='3M Company provides diversified technology services in the United States and internationally. The company operates through four segments: Safety and Industrial; Transportation and Electronics; Health Care; and Consumer. The Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; closure systems for personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection solu

In [61]:
## show() method displays the first 20 rows of the DataFrame
sp500_info_df.show(5)


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [58]:
## save_as_table() method saves the DataFrame to a table in the database
#sp500_info_df.save_as_table("investment_db.investment_schema.sp500_info_copy")

In [62]:
## to_pandas() method returns a Pandas DataFrame
sp500_info_pd = sp500_info_df.to_pandas()
sp500_info_pd.tail()

,SYMBOL,ADDRESS1,CITY,STATE,ZIP,COUNTRY,PHONE,WEBSITE,INDUSTRY,INDUSTRYKEY,...,PEGRATIO,TARGETHIGHPRICE,TARGETLOWPRICE,TARGETMEANPRICE,TARGETMEDIANPRICE,RECOMMENDATIONMEAN,NUMBEROFANALYSTOPINIONS,EARNINGSGROWTH,FAX,INDUSTRYSYMBOL
498,YUM,1441 Gardiner Lane,Louisville,KY,40213,United States,502 874 8300,https://www.yum.com,Restaurants,None,...,1.79,160.0,116.0,138.10,136.0,2.4,21.0,0.281,None,None
499,ZBRA,3 Overlook Point,Lincolnshire,IL,60069,United States,847 634 6700,https://www.zebra.com,Communication Equipment,None,...,6.08,340.0,183.0,265.67,275.0,2.4,9.0,NaN,None,None
500,ZBH,345 East Main Street,Warsaw,IN,46580,United States,574 373 3121,https://www.zimmerbiomet.com,Medical Devices,None,...,2.28,150.0,99.0,129.38,130.0,2.6,24.0,-0.163,None,None
501,ZION,One South Main Street,Salt Lake City,UT,84133-1109,United States,801 844 7637,https://www.zionsbancorporation.com,Banks—Regional,None,...,-0.31,48.0,32.0,37.95,37.5,2.7,20.0,-0.191,None,None
502,ZTS,10 Sylvan Way,Parsippany,NJ,07054,United States,973 822 7000,https://www.zoetis.com,Drug Manufacturers—Specialty & Generic,None,...,3.53,255.0,190.0,216.18,215.0,1.9,11.0,0.142,None,None


## Saving Data to a Table


In [ ]:
## .write.mode("overwrite") method overwrites the table in the database
sp500_info_df.write.mode("overwrite").save_as_table("table1")

## Creating a View from a DataFrame

In [66]:
## database and schema are only required if you are not working in the context of a database and schema
database = "investment_db"
schema = "investment_schema"
view_name = "my_view"
sp500_info_df.create_or_replace_view(f"{database}.{schema}.{view_name}")

[Row(status='View MY_VIEW successfully created.')]

## Working with Semi-structured (JSON) data - COME BACK TO THIS

In [88]:
# sp500_info_df = session.table("investment_db.investment_schema.sp500_info")
# #sp500_info_df.select(col('COMPANYOFFICERS')).show()
# flattened_df = sp500_info_df.join_table_function("flatten", sp500_info_df["COMPANYOFFICERS"]).select(col("value:name").as_("officer_name"))
# flattened_df.show()


SnowparkSQLException: (1304): 01b10891-0004-df70-0002-9957000211fe: 001007 (22023): SQL compilation error:
invalid type [VARCHAR(16777216)] for parameter '1'